# Postprocesamiento y Limpieza de Datos
Este código realiza el postprocesamiento y limpieza de datos extraídos de Resoluciones de Calificación Ambiental (RCA). Su propósito principal es:
- Importación y preparación de datos
- Eliminación de datos irrelevantes
- Eliminación de duplicados
- Clasificación y corrección
- Exportación de datos limpios

## Importar y definir directorios

In [3]:
# Librerías estándar de Python
import ast
import copy
import io
import json
import os
import re
import shutil
import time
import uuid
import xml.etree.ElementTree as ET
from enum import Enum
from html import unescape
from itertools import combinations, permutations, zip_longest
from time import sleep

# Manejo de documentos y archivos
import docx
import openpyxl
import pdfplumber
from PyPDF2 import PdfReader
import tabula
from tabula.io import read_pdf

# Web scraping y requests
import requests
from bs4 import BeautifulSoupimport matplotlib.pyplot as plt
# Minimal test for yfiles_jupyter_graphs
import networkx as nx
from yfiles_jupyter_graphs import GraphWidget
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import plot
import colorsys
        from yfiles_jupyter_graphs import GraphWidget
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from typing import Dict
from random import random
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import font_manager
from adjustText import adjust_text

    # Crear leyenda para los ministerios con tamaño más grande
    from matplotlib.lines import Line2D

from networkx import florentine_families_graph
from yfiles_jupyter_graphs import GraphWidget
from typing import Dict
from yfiles_jupyter_graphs import GraphWidget
from IPython.display import display
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Machine Learning / IA / Embeddings
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from transformers import AutoTokenizer

# APIs y entornos
from dotenv import load_dotenv
import openai
from openai import OpenAI, AzureOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import tiktoken

# Pydantic y validación
from pydantic import BaseModel, Field, ValidationError, model_validator, constr, conlist
from typing import List, Optional, Literal, Annotated

# Manejo de datos
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from tqdm import tqdm

# Fuzzy Matching
from fuzzywuzzy import fuzz
from thefuzz import fuzz, process

### Establecer Directorios

In [ ]:
sector = f"energia"
path = os.path.join(os.environ['USERPROFILE'], "RUTA", f"archivos_{sector}")
load_dotenv()
api_key_openai = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(
    api_key=api_key_openai
)

In [ ]:
df = pd.read_excel(os.path.join(path, "obligaciones/obligaciones.xlsx"))

## Eliminar Errores

In [ ]:
condicion_eliminar = (df['justificacion'].str.strip().str.len() <= 15) & (df['resumen'].str.strip().str.len() <= 15)
df_cleaned = df[~condicion_eliminar]
print(f"se eliminó un {(len(df) - len(df_cleaned))*100/len(df)} % de los datos")

In [ ]:
# Lista de términos prohibidos
forbidden_terms = [
    "no identificado", "no aplica", "no se comprometen límites",
    "No se especifica", "No se especifica en el texto",
    "no identificado y nan", "no indicado", "no especificado"
]

# Función para verificar similitud difusa
def is_similar(value, terms, threshold=95):
    if pd.isna(value):
        return True
    value_str = str(value).strip().lower()
    if len(value_str) > 50:
        return False
    if len(value_str) <= 10:
        return True
    return any(fuzz.ratio(value_str, term.lower()) >= threshold for term in terms)

df_cleaned = df_cleaned[df_cleaned['seccion'].isin(['pas', 'contingencias_emergencias']) |
~(df_cleaned['resumen'].apply(lambda x: is_similar(x, forbidden_terms)) | 
  df_cleaned['justificacion'].apply(lambda x: is_similar(x, forbidden_terms)))]

print(f"se eliminó un {(len(df) - len(df_cleaned))*100/len(df)} % de los datos")

## Duplicados Exactos

In [ ]:
len_original_df = len(df_cleaned)
df_cleaned['numero_tabla'] = df_cleaned['numero_tabla'].astype(str) 
df_cleaned['numero_tabla_grupo'] = df_cleaned['numero_tabla'].str[:15]

cond_just_valid = (df_cleaned['justificacion'] != "") & (df_cleaned['justificacion'].notna())
df_cleaned = pd.concat([
    df_cleaned[cond_just_valid].drop_duplicates(
        subset=['cell', 'seccion', 'numero_tabla_grupo', 'justificacion'],
        keep='first'
    ),
    df_cleaned[~cond_just_valid]
], ignore_index=True)


cond_res_valid = (df_cleaned['resumen'] != "") & (df_cleaned['resumen'].notna())
df_cleaned = pd.concat([
    df_cleaned[cond_res_valid].drop_duplicates(
        subset=['cell', 'seccion', 'numero_tabla_grupo', 'resumen'],
        keep='first'
    ),
    df_cleaned[~cond_res_valid]
], ignore_index=True)


indices_a_eliminar = set()
for (seccion_val, cell_val), grupo in df_cleaned.groupby(['seccion', 'cell']):
    no_identificados = grupo[grupo['numero_tabla_grupo'] == 'no identificado']
    identificados = grupo[grupo['numero_tabla_grupo'] != 'no identificado']

    if not no_identificados.empty and not identificados.empty:
        justificaciones_identificadas = set(identificados['justificacion'].dropna())
        resumenes_identificados = set(identificados['resumen'].dropna())

        for idx, fila_ni in no_identificados.iterrows():
            if (fila_ni['justificacion'] in justificaciones_identificadas or fila_ni['resumen'] in resumenes_identificados):
                indices_a_eliminar.add(idx)

# Eliminar las filas marcadas de df_cleaned
if indices_a_eliminar:
    df_cleaned = df_cleaned.drop(index=list(indices_a_eliminar))

# Cálculo de Porcentaje Eliminado (Final)
porcentaje_eliminado_total = (len_original_df - len(df_cleaned)) * 100 / len_original_df if len_original_df > 0 else 0
print(f"\n### Proceso de Limpieza Completado ###")
print(f"Porcentaje TOTAL de datos eliminados: {porcentaje_eliminado_total:.2f} %")
print(f"Tamaño original del DataFrame: {len_original_df} filas")
print(f"Tamaño final del DataFrame: {len(df_cleaned)} filas")

## Separar según Decretos PAS

In [ ]:
def extract_articulos_dict(texto):

    regex = r"Artículo (\d+)\.-"
    articulos = re.split(regex, texto)

    # Asegurarse de que el primer elemento no cause problemas
    if articulos and articulos[0]:
        articulos = articulos[1:]
    elif articulos:
        articulos = articulos[1:]

    articulos_dict = {}
    for i in range(0, len(articulos) - 1, 2):
        numero_articulo = articulos[i]
        contenido = articulos[i + 1].strip()
        articulos_dict[numero_articulo] = contenido

    return articulos_dict

def agregar_contenido_articulo(df, articulos_dict):

    df['contenido_articulo'] = df.apply(
        lambda row: articulos_dict.get(row['numero_fuente'].strip("['']"), pd.NA)
        if row['seccion'] == 'pas'
        else pd.NA,
        axis=1,
    )
    return df

# Ruta al archivo Word
word_file_path = os.path.join(os.environ['USERPROFILE'], "RUTA", "PAS.docx")

# Leer el contenido del archivo Word
doc = docx.Document(word_file_path)
full_text = []
for paragraph in doc.paragraphs:
    full_text.append(paragraph.text)
texto = '\n'.join(full_text)

# Procesar el texto para extraer los artículos en un diccionario
articulos_dict = extract_articulos_dict(texto)

# Agregar la columna 'contenido_articulo'
df_cleaned = agregar_contenido_articulo(df_cleaned, articulos_dict)  

def get_embedding(text, deployment_name):
    """Obtiene el embedding para un texto dado."""
    response = client.embeddings.create(input=[text], model=deployment_name)
    return np.array(response.data[0].embedding)

def cosine_similarity(emb1, emb2):
    """Calcula la similitud coseno entre dos embeddings."""
    return dot(emb1, emb2) / (norm(emb1) * norm(emb2)) if (norm(emb1) * norm(emb2)) != 0 else 0

lista_de_terminos = df_cleaned.sin_condiciones.value_counts(normalize=True).head(n=30).index.tolist()


def limpiar_condiciones_avanzado(texto_original):
    if pd.isna(texto_original):
        return None 

    texto = str(texto_original)
    texto = texto.replace(u'\xa0', ' ')  
    texto = texto.replace('\n', ' ')    
    texto = texto.lstrip('- ')          
    texto = re.sub(r'\s+', ' ', texto).strip() 
    texto_lower = texto.lower()
    s_c_normalizado = "S/C" 

    terminos_exactos_para_sc = {
        "s/c": s_c_normalizado,
        "s/c.": s_c_normalizado,
        "s/c,": s_c_normalizado,
        "s / c": s_c_normalizado,
        "s / c.": s_c_normalizado,
        "s / c,": s_c_normalizado,
        "no identificado": s_c_normalizado,
        "no hay.": s_c_normalizado,
        "no hay": s_c_normalizado,
        "ninguna": s_c_normalizado,
        "ninguna.": s_c_normalizado,
        "no": s_c_normalizado, 
        "s/c, s/c": s_c_normalizado,
        "s/c, s/c, s/c": s_c_normalizado,
    }
    if texto_lower in terminos_exactos_para_sc:
        return terminos_exactos_para_sc[texto_lower]

    patrones_frases_sc_completas = [
        r"no se especifican acciones de emergencia en el texto proporcionado\.?",
        r"no se especifican acciones de emergencia en el texto\.?",
        r"las emisiones producidas por el proyecto se encuentran debidamente abordadas por las medidas de control propuestas por el titular del proyecto y no generarán riesgo a la salud de la población del área de influencia del proyecto debido a la cantidad y calidad de efluentes, emisiones o residuos\.?",
        r"las emisiones producidas por el proyecto se encuentran debidamente abordadas por las medidas de control propuestas por el titular del proyecto y no generarán riesgo a la salud de la población del área de influencia del proyecto debido a la cantidad y calidad de efluentes, emisiones o residuos, tal como se puede apreciar en el punto \d+\.\d+\.\d+ del ice\.?",
        r"el requisito para su otorgamiento consiste en que las condiciones de saneamiento y seguridad eviten un riesgo a la salud de la población\.?",
        r"la seremi de salud, órgano del estado que otorga dicho permiso, se pronuncia conforme a los antecedentes presentados, mediante oficio ord\. no \d+ de fecha \d{1,2} de \w+ de \d{4}\.?",
        r"no existen condiciones o exigencias asociadas a este permiso\. pronunciamiento del órgano competente: mediante ord\. n°\d+ de fecha \d{1,2}/\d{1,2}/\d{4}, la seremi de salud, región de [\w\s]+, se pronunció conforme\.?",
        r"sin condiciones\.?",
        r"no se establecen condiciones\.?",
        r"no se establecieron condiciones\.?",
        r"no existen condiciones\.?",
        r"sin restricciones\.?",
        r"no hay condiciones\.?",
        r"no aplican condiciones\.?",
        r"ninguna condición\.?"
    ]
    for patron in patrones_frases_sc_completas:
        if re.fullmatch(patron, texto_lower):
            return s_c_normalizado

    match_sc_al_inicio = re.match(r"^(s\s*/\s*c\b'*[\.,]?)\s*", texto_lower)

    if match_sc_al_inicio:
        longitud_prefijo_sc = len(match_sc_al_inicio.group(0))
        contenido_despues_sc = texto[longitud_prefijo_sc:].strip()

        patrones_ruido_despues_de_sc = [
            r"^,'frecuencia':'eventual'[\.,]?\s*",
            r"^\['\"\]?,\s*\['\"\]?frecuencia\['\"\]?\s*:\s*\['\"\]?eventual\['\"\]?\.?,?\s*",
            r"^\(?(no\s+hay)\)?\.?,?\s*",
            r"^frecuencia\s*:\s*eventual\.?,?\s*",
            r"^frecuencia\s*eventual\.?,?\s*",
            r"^frecuencia\.?,?\s*",
        ]
        
        contenido_provisional = contenido_despues_sc
        for patron_ruido in patrones_ruido_despues_de_sc:
            contenido_provisional = re.sub(patron_ruido, "", contenido_provisional, count=1, flags=re.IGNORECASE).strip()
        
        contenido_final_despues_sc = contenido_provisional.lstrip(',. ').strip()

        if not contenido_final_despues_sc:
            return s_c_normalizado
        else:
            if contenido_final_despues_sc.lower() in ["s/c", "s/c.", "s/c,"]:
                return s_c_normalizado
            return f"{s_c_normalizado}, {contenido_final_despues_sc}"

    patrones_genericos_sc_embebidos = [
        r"sin condiciones", r"no se establecen condiciones", r"no se establecieron condiciones",
        r"no existen condiciones", r"sin restricciones", r"no hay condiciones",
        r"no aplican condiciones", r"ninguna condición"
    ]
    for patron in patrones_genericos_sc_embebidos:
        if re.search(patron, texto_lower):
            if len(texto_lower) < 50 and not re.search(r"(pero|excepto|salvo|aunque|no obstante|sin embargo)", texto_lower):
                return s_c_normalizado
            break 
    return texto

df_cleaned['sin_condiciones'] = df_cleaned['sin_condiciones'].apply(limpiar_condiciones_avanzado)
df_cleaned['similitud_fuzz'] = None
df_cleaned['similitud_embedding'] = None
df_cleaned['sin_condiciones_final'] = None 

for index, row in df_cleaned.iterrows():
    if isinstance(row['sin_condiciones'], str) and isinstance(row['contenido_articulo'], str):
        print(row['cell'])
        fuzz_similarity = fuzz.partial_ratio(row['sin_condiciones'], row['contenido_articulo']) / 100.0 
        df_cleaned.loc[index, 'similitud_fuzz'] = fuzz_similarity
        try:
            embedding_sin_condiciones = get_embedding(row['sin_condiciones'], "text-embedding-3-large")
            embedding_contenido_articulo = get_embedding(row['contenido_articulo'], "text-embedding-3-large")
            if embedding_sin_condiciones is not None and embedding_contenido_articulo is not None:
                similarity = cosine_similarity(embedding_contenido_articulo, embedding_sin_condiciones)
                df_cleaned.loc[index, 'similitud_embedding'] = similarity
                print(fuzz_similarity, similarity)
                if similarity > 0.60 or fuzz_similarity > 0.60:
                    df_cleaned.loc[index, 'sin_condiciones_final'] = 'Se se establecen condiciones adicionales distintas a las del artículo'
                else:
                    df_cleaned.loc[index, 'sin_condiciones_final'] = 'No se establecen condiciones adicionales distintas a las del artículo' 
            else:
                df_cleaned.loc[index, 'similitud_embedding'] = None
                df_cleaned.loc[index, 'sin_condiciones_final'] = 'No se establecen condiciones'
        except Exception as e:
            print(f"Error calculating embedding similarity for row {index}: {e}")
            df_cleaned.loc[index, 'similitud_embedding'] = None
            df_cleaned.loc[index, 'sin_condiciones_final'] = 'No se establecen condiciones'
    else:
        df_cleaned.loc[index, 'sin_condiciones_final'] = 'No se establecen condiciones' 

print(df_cleaned)


In [ ]:
df_cleaned.to_excel(os.path.join(path, "df_obligaciones_pre_filtradas_2.xlsx"), index = False)

## Eliminar Duplicados Semánticos

### Detección Semántica de Duplicados

In [ ]:
df_X = pd.read_excel(os.path.join(path, "df_obligaciones_pre_filtradas_2.xlsx"))

# Variables globales para seguimiento de costos y tokens de embeddings
embedding_costs = {"text-embedding-3-large": 0.00013}
total_embedding_tokens = 0
total_embedding_cost = 0.0
embedding_calls = 0

# Función para contar tokens para OpenAI
def count_tokens_embeddings(text, model="text-embedding-3-large"):
    """Cuenta los tokens para el modelo de embeddings especificado."""
    # Encoding correcto según el modelo
    if "text-embedding-3" in model:
        encoding = tiktoken.get_encoding("cl100k_base")
    else: 
        encoding = tiktoken.get_encoding("p50k_base")
    
    token_list = encoding.encode(text)
    return len(token_list)


# Método 2: Usando directamente la biblioteca de OpenAI
def similitud_embeddings_direct(texto1, texto2, api_key, endpoint, deployment_name, api_version="2024-02-01", track_usage=True):
    global total_embedding_tokens, total_embedding_cost, embedding_calls
    
    if not isinstance(texto1, str) or not isinstance(texto2, str):
        return 0, None
    
    model = "text-embedding-3-large"
    
    # Contar tokens antes de enviar a la API
    tokens_texto1 = count_tokens_embeddings(texto1, model) if track_usage else 0
    tokens_texto2 = count_tokens_embeddings(texto2, model) if track_usage else 0
    total_tokens = tokens_texto1 + tokens_texto2
    
    # Calcular costo
    if track_usage:
        cost_rate = embedding_costs.get(model, 0.0001)  
        cost = (total_tokens / 1000) * cost_rate
        
        # Actualizar contadores globales
        total_embedding_tokens += total_tokens
        total_embedding_cost += cost
        embedding_calls += 1
    
    try:
        
        # Obtener embeddings para ambos textos
        response1 = client.embeddings.create(
            input=texto1,
            model=deployment_name
        )
        def get_embedding(text, deployment_name):
            """Obtiene el embedding para un texto dado."""
            response = client.embeddings.create(input=[text], model=deployment_name)
            return np.array(response.data[0].embedding)

        
        response2 = client.embeddings.create(
            input=texto2,
            model=deployment_name
        )
        
        # Extraer los vectores de embedding
        embedding1 = np.array(response1.data[0].embedding)
        embedding2 = np.array(response2.data[0].embedding)
        
        # Calcular similitud coseno
        cos_sim = dot(embedding1, embedding2) / (norm(embedding1) * norm(embedding2))
        cos_sim = max(min(cos_sim, 1.0), 0.0)
        
        # Obtener tokens reales de la respuesta (si está disponible)
        if hasattr(response1, 'usage') and hasattr(response1.usage, 'total_tokens'):
            tokens_real1 = response1.usage.total_tokens
        else:
            tokens_real1 = tokens_texto1
            
        if hasattr(response2, 'usage') and hasattr(response2.usage, 'total_tokens'):
            tokens_real2 = response2.usage.total_tokens
        else:
            tokens_real2 = tokens_texto2
            
        total_tokens_real = tokens_real1 + tokens_real2
        
        # Actualizar costo con tokens reales
        if track_usage and total_tokens != total_tokens_real:
            # Ajustar contadores globales si hay diferencia
            total_embedding_tokens = total_embedding_tokens - total_tokens + total_tokens_real
            cost_real = (total_tokens_real / 1000) * cost_rate
            total_embedding_cost = total_embedding_cost - cost + cost_real
            cost = cost_real
        
        # Crear información de uso
        if track_usage:
            usage_info = {
                'modelo': model,
                'deployment': deployment_name,
                'tokens_texto1': tokens_real1,
                'tokens_texto2': tokens_real2,
                'total_tokens': total_tokens_real,
                'costo': f"{cost:.6f} USD",
                'total_acumulado': {
                    'llamadas': embedding_calls,
                    'tokens': total_embedding_tokens,
                    'costo': f"{total_embedding_cost:.6f} USD"
                }
            }
        else:
            usage_info = None
        
        return cos_sim * 100, usage_info
    
    except Exception as e:
        print(f"Error al calcular embeddings directamente: {e}")
        if track_usage:
            usage_info = {
                'modelo': model,
                'deployment': deployment_name,
                'tokens_texto1': tokens_texto1,
                'tokens_texto2': tokens_texto2,
                'total_tokens': total_tokens,
                'costo': f"{cost:.6f} USD",
                'error': str(e),
                'total_acumulado': {
                    'llamadas': embedding_calls,
                    'tokens': total_embedding_tokens,
                    'costo': f"{total_embedding_cost:.6f} USD"
                }
            }
            return 0, usage_info
        return 0, None

def similitud_embeddings(texto1, texto2, api_key, endpoint, deployment_name, api_version="2024-02-01", use_langchain=False, track_usage=True):
    if use_langchain:
        return "ERROR"
    else:
        return similitud_embeddings_direct(texto1, texto2, api_key, endpoint, deployment_name, api_version, track_usage)

# Código modificado para procesar el DataFrame completo
def procesar_similitudes(df_filtered, api_key, endpoint, deployment_name, api_version="2024-02-01", 
                         umbral_similitud=85, use_langchain=False, path=".", output_filename="obligaciones_filtradas.xlsx"):
    codigo_par_similar = {}
    total_comparaciones = 0
    pares_similares = 0
    
    grupos = df_filtered.groupby(['cell', 'seccion', 'numero_tabla', 'orden_obligacion', 
                                  'componente_materia', 'fase', 'tipo_obligacion', 
                                  'frecuencia', 'fuente', 'objeto'])
    
    total_grupos = len(list(grupos))
    print(f"Total de grupos a procesar: {total_grupos}")
    
    # Reiniciar el iterador de grupos
    grupos = df_filtered.groupby(['cell', 'seccion', 'numero_tabla', 'orden_obligacion', 
                                 'componente_materia', 'fase', 'tipo_obligacion', 
                                 'frecuencia', 'fuente', 'objeto'])
    
    grupo_contador = 0
    guardar_cada_n_pares = 10 
    pares_guardados_desde_ultimo = 0
    for (cell, seccion, numero_tabla, orden_obligacion, componente_materia, fase, 
         tipo_obligacion, frecuencia, fuente, objeto), group in grupos:
        grupo_contador += 1
        num_frases = len(group['resumen'])
        if num_frases <= 1:
            print(f"Grupo {grupo_contador}/{total_grupos} ({cell}, {seccion}, {numero_tabla}): Solo tiene 1 frase, omitiendo.")
            continue
        
        pares_frases = list(combinations(group['resumen'], 2))
        total_comparaciones += len(pares_frases)
        
        print(f"Procesando grupo {grupo_contador}/{total_grupos} ({cell}, {seccion}, {numero_tabla}): {len(pares_frases)} comparaciones")
        
        for i, (frase1, frase2) in enumerate(pares_frases, 1):
            # Mostrar progreso periódicamente
            if i % 10 == 0 or i == len(pares_frases):
                print(f"  Progreso: {i}/{len(pares_frases)} comparaciones")
            
            # Calcular similitud con seguimiento de uso
            similitud, usage_info = similitud_embeddings(
                frase1, frase2, 
                api_key, endpoint, deployment_name, api_version,
                use_langchain=use_langchain
            )
            
            # Mostrar información de uso cada cierto número de llamadas
            if usage_info and embedding_calls % 50 == 0:
                acumulado = usage_info['total_acumulado']
                print(f"\nEstadísticas de uso de embeddings:")
                print(f"  Llamadas totales: {acumulado['llamadas']}")
                print(f"  Tokens totales: {acumulado['tokens']}")
                print(f"  Costo total: {acumulado['costo']}")
                print(f"  Comparaciones: {total_comparaciones}, Pares similares: {pares_similares}")
                if total_comparaciones > 0:
                    print(f"  Porcentaje de similitud: {(pares_similares/total_comparaciones*100):.2f}%")
                print("-" * 40)
            
            # Si la similitud es mayor al umbral, registrarla
            if similitud > umbral_similitud:
                pares_similares += 1
                pares_guardados_desde_ultimo += 1
                print(f"Similitud entre el par {i}, {cell}, {seccion}, {numero_tabla} : {similitud:.2f}%")
                print(f"Frase 1: {frase1}")
                print(f"Frase 2: {frase2}")
                if usage_info:
                    print(f"Tokens: {usage_info['total_tokens']}, Costo: {usage_info['costo']}")
                print("-" * 40)
                
                idx_frase1 = group[group['resumen'] == frase1].index[0]
                idx_frase2 = group[group['resumen'] == frase2].index[0]
                codigo_par_similar[idx_frase1] = similitud
                codigo_par_similar[idx_frase2] = similitud
            
                # Agregar resultados y guardar temporalmente si se cumple condición
                df_filtered['codigo_par_similar'] = df_filtered.index.map(codigo_par_similar).fillna('N/A')
            
                if pares_guardados_desde_ultimo >= guardar_cada_n_pares:
                    output_path = os.path.join(path, output_filename)
                    df_filtered.to_excel(output_path, index=False)
                    print(f"[Guardado parcial] Archivo actualizado con {pares_similares} pares similares en: {output_path}")
                    pares_guardados_desde_ultimo = 0

    print("\n========== RESUMEN FINAL DE USO DE EMBEDDINGS ==========")
    print(f"Total de grupos procesados: {grupo_contador}")
    print(f"Total de comparaciones realizadas: {total_comparaciones}")
    print(f"Total de pares similares encontrados: {pares_similares}")
    if total_comparaciones > 0:
        print(f"Porcentaje de similitud: {(pares_similares/total_comparaciones*100):.2f}%")
    print(f"Total de llamadas a la API: {embedding_calls}")
    print(f"Total de tokens procesados: {total_embedding_tokens}")
    print(f"Costo total estimado: ${total_embedding_cost:.6f} USD")
    print("=======================================================")
    
    df_filtered['codigo_par_similar'] = df_filtered.index.map(codigo_par_similar).fillna('N/A')
    
    # Guardar el resultado
    output_path = os.path.join(path, output_filename)
    df_filtered.to_excel(output_path, index=False)
    print(f"Archivo guardado en: {output_path}")
    
    return df_filtered, {
        'comparaciones': total_comparaciones,
        'pares_similares': pares_similares,
        'llamadas_api': embedding_calls,
        'tokens_totales': total_embedding_tokens,
        'costo_total': total_embedding_cost
    }

# Ejecutar el proceso
df_X, estadisticas = procesar_similitudes(
    df_filtered=df_X,
    api_key=api_key_openai,
    endpoint=endpoint_openai,
    deployment_name="text-embedding-3-large",
    api_version=api_version_openai,
    umbral_similitud=75,
    use_langchain=False,  
    path=".",
    output_filename=os.path.join(path, "df_obligaciones_pre_filtradas_3.xlsx")
)


In [ ]:
system_instructions = {
    "obligaciones": {
        "categorizar": """
        Evalúa si dos obligaciones son semánticamente equivalentes (TRUE) o no (FALSE) basándote en las acciones fundamentales que exigen.

        ✓ EQUIVALENTES (TRUE) cuando:
        • Las acciones principales o resultados exigidos son fundamentalmente los mismos
        • Una es reformulación, resumen o versión más detallada de la otra, manteniendo la esencia
        • Las diferencias son solo de redacción, sinónimos o formato, sin cambiar sustanciamente la acción detallada
        • La acción resultante o propósito principal coincide, aunque varíe la descripción del proceso

        ✗ NO EQUIVALENTES (FALSE) cuando:
        • Exigen acciones claramente distintas, complementarias, adicionales o faltantes entre sí
        • Se refieren a objetos, sujetos o contextos diferentes que implican acciones distintas
        • Difieren en alcance, magnitud, frecuencia o parámetros específicos que afectan la implementación
        • Una obligación es significativamente más amplia o restrictiva que la otra

        Responde únicamente con un objeto JSON que contenga la clave 'equivalent' con valor true o false.
        """,

        "eg_input_1": '''Obligación 1: "Obtención del permiso ambiental sectorial N° 140 otorgado por la Seremi de Salud respectiva."
Obligación 2: "Obtención del permiso ambiental sectorial N° 142 otorgado por la Seremi de Salud respectiva."''',
        "eg_output_1": '''{"equivalent": false}''', 

        "eg_input_2": '''Obligación 1: "Presentar el Plan de Manejo Forestal a CONAF y obtener su aprobación."
Obligación 2: "Aprobación de Plan de Manejo Forestal por CONAF."''',
        "eg_output_2": '''{"equivalent": true}''',

        "eg_input_3": '''Obligación 1: "Transportar los materiales en camiones con la carga cubierta."
Obligación 2: "Transporte de materiales en camiones con la carga cubierta mediante el empleo de lona."''',
        "eg_output_3": '''{"equivalent": true}''', 

        "eg_input_4": '''Obligación 1: "Realizar monitoreo trimestral de calidad del agua y presentar informes."
Obligación 2: "Monitorear semestralmente los niveles de contaminantes en agua y reportar resultados."''',
        "eg_output_4": '''{"equivalent": false}''', 

        "eg_input_5": '''Obligación 1: "Replantar especies nativas en proporción de 3:1 por cada árbol removido."
Obligación 2: "Replantar especies nativas en proporción de 2:1 por cada árbol removido."''',
        "eg_output_5": '''{"equivalent": false}''', 

        "eg_input_6": '''Obligación 1: "Realizar charlas de inducción en arqueología, resaltando importancia de la preservación del patrimonio prehispánico a trabajadores antes de excavación."
Obligación 2: "Dar una charla de inducción sobre arqueología a los trabajadores antes de iniciar la obra."''',
        "eg_output_6": '''{"equivalent": true}''', 
    }
}

class ObligacionesResponse(BaseModel):
    equivalent: bool = Field(description="Booleano que es TRUE cuando las obligaciones son equivalentes y FALSE en caso contrario")

response_format = ObligacionesResponse

def comparar_obligaciones(client, obligacion1, obligacion2):
    initial_messages = [
        {"role": "user", "content": system_instructions["obligaciones"]["eg_input_1"]},
        {"role": "assistant", "content": system_instructions["obligaciones"]["eg_output_1"]},
        {"role": "user", "content": system_instructions["obligaciones"]["eg_input_2"]},
        {"role": "assistant", "content": system_instructions["obligaciones"]["eg_output_2"]}
    ]
    
    input_text = f'Obligación 1: "{obligacion1}"\nObligación 2: "{obligacion2}"'
    
    messages = [{"role": "system", "content": system_instructions["obligaciones"]["categorizar"]}]
    messages.extend(initial_messages)
    messages.append({"role": "user", "content": input_text})
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0,
        response_format=response_format,
        messages=messages,
    )
    
    response_content = completion.choices[0].message.content
    result = json.loads(response_content)
    
    return result["equivalent"]

In [ ]:
def procesar_y_deduplicar_obligaciones(df, client):
    df_modificado = df.copy()
    df_modificado['eliminacion_duplicados_semanticos'] = False
    indices_a_eliminar = []
    indices_con_duplicado_eliminado = []
    total_grupos_procesados = 0
    pares_equivalentes = 0
    filas_eliminadas = 0
    
    count = 0 
    for id_grupo, grupo in df.groupby(['codigo_par_similar', 'cell', 'seccion']):
        if len(grupo) != 2:            
            print(f"Advertencia: El grupo identificado por {id_grupo} tiene {len(grupo)} obligaciones, se esperaban 2. Se omitirá este grupo.")
            continue

        count = count +1 
        total_grupos_procesados += 1
        
        # Extraer las dos filas
        fila1 = grupo.iloc[0]
        fila2 = grupo.iloc[1]
        obligacion1_texto = f"{fila1.get('resumen', '')} {fila1.get('justificacion', '')}".strip()
        obligacion2_texto = f"{fila2.get('resumen', '')} {fila2.get('justificacion', '')}".strip()
        if not obligacion1_texto or not obligacion2_texto:
            print(f"Advertencia: Para el grupo {id_grupo}, una o ambas obligaciones resultaron vacías. Se omitirá.")
            print(f"Texto Obligación 1: '{obligacion1_texto}'")
            print(f"Texto Obligación 2: '{obligacion2_texto}'")
            continue
        
        print(f"\n=== COMPARACIÓN {count}/9567, PARA GRUPO ID: {id_grupo} ===")
        print(f"Obligación 1 (Índice {fila1.name}): {obligacion1_texto}")
        print(f"Obligación 2 (Índice {fila2.name}): {obligacion2_texto}")
        
        try:
            equivalente = comparar_obligaciones(client, obligacion1_texto, obligacion2_texto)
            resultado_texto = 'Equivalentes' if equivalente else 'No equivalentes'
            print(f"Resultado: {resultado_texto}")
            
            # Si son equivalentes, marcar una para eliminar y la otra para señalar como duplicado eliminado
            if equivalente:
                pares_equivalentes += 1
                indice_a_eliminar = fila2.name
                indice_a_marcar = fila1.name
                
                if indice_a_eliminar not in indices_a_eliminar: 
                    indices_a_eliminar.append(indice_a_eliminar)
                if indice_a_marcar not in indices_con_duplicado_eliminado: 
                     indices_con_duplicado_eliminado.append(indice_a_marcar)
                
                print(f"    → Se marcará para eliminar la fila con índice original {indice_a_eliminar}")
                print(f"    → Se marcará la fila con índice original {indice_a_marcar} como TRUE en 'eliminacion_duplicados_semanticos'")
                
        except Exception as e:
            print(f"Error al procesar el grupo {id_grupo}: {str(e)}")
    
    final_indices_a_marcar = [idx for idx in indices_con_duplicado_eliminado if idx not in indices_a_eliminar]
    for indice in final_indices_a_marcar:
        if indice in df_modificado.index: 
             df_modificado.loc[indice, 'eliminacion_duplicados_semanticos'] = True
        else:
            print(f"Advertencia: Índice {indice} para marcar como TRUE no encontrado en df_modificado. Pudo haber sido marcado para eliminación.")

    df_limpio = df_modificado.drop(index=list(set(indices_a_eliminar))) 
    filas_eliminadas = len(list(set(indices_a_eliminar)))

    print("\n=== RESUMEN DE DEDUPLICACIÓN ===")
    print(f"Total de grupos de dos obligaciones procesados: {total_grupos_procesados}")
    print(f"Pares equivalentes encontrados: {pares_equivalentes}")
    print(f"Filas eliminadas: {filas_eliminadas}")

    filas_marcadas_true_final = df_limpio['eliminacion_duplicados_semanticos'].sum()
    print(f"Filas marcadas como TRUE en 'eliminacion_duplicados_semanticos' (en df limpio): {filas_marcadas_true_final}")
    print(f"Filas en el DataFrame original: {len(df)}")
    print(f"Filas en el DataFrame limpio: {len(df_limpio)}")
    
    # Mostrar distribución de valores en la nueva columna
    if not df_limpio.empty:
        valor_counts = df_limpio['eliminacion_duplicados_semanticos'].value_counts()
        print("\nDistribución de valores en 'eliminacion_duplicados_semanticos' (en df limpio):")
        print(f"TRUE: {valor_counts.get(True, 0)}")
        print(f"FALSE: {valor_counts.get(False, 0)}")
    else:
        print("\nEl DataFrame limpio está vacío.")
        
    return df_limpio

In [ ]:
if __name__ == "__main__":
    client = openai.OpenAI(
        api_key=api_key_openai
    )
    df_sin_duplicados = procesar_y_deduplicar_obligaciones(df_X, client)
df_sin_duplicados.to_excel(os.path.join(path, "obligaciones_final_final.xlsx"), index = False)

### Eliminar "No Identificados" que tengan misma idea semántica que "Identificados"

In [ ]:
df_sin_duplicados = pd.read_excel(os.path.join(path, "obligaciones_final_final.xlsx"))

In [ ]:
df_sin_duplicados.seccion.value_counts()

In [ ]:
def get_embeddings_batch(texts, client, deployment_name, model="text-embedding-3-large"):
    global total_embedding_tokens, total_embedding_cost, embedding_calls
    
    valid_texts = [text for text in texts if isinstance(text, str) and text.strip()]
    if not valid_texts:
        return {}

    # Contar tokens y calcular costo
    tokens_count = sum(count_tokens_embeddings(text, model) for text in valid_texts)
    cost_rate = embedding_costs.get(model, 0.00013)
    cost = (tokens_count / 1000) * cost_rate
    
    # Actualizar contadores globales
    total_embedding_tokens += tokens_count
    total_embedding_cost += cost
    embedding_calls += 1
    
    try:
        response = client.embeddings.create(
            input=valid_texts,
            model=deployment_name
        )
        # Crear el diccionario de cache: {texto: embedding}
        embeddings_cache = {text: np.array(data.embedding) for text, data in zip(valid_texts, response.data)}
        return embeddings_cache
        
    except Exception as e:
        print(f"Error al obtener embeddings en lote: {e}")
        return {}


def marcar_similitudes_no_identificadas(df, client, deployment_name, umbral_similitud=75, output_dir=".", output_filename="obligaciones_marcadas.xlsx"):
    """
    Busca similitudes usando un método optimizado de batching para los embeddings.
    """
    print("Iniciando proceso optimizado de detección de similitudes...")
    
    df_copy = df.copy()
    df_copy['es_duplicado_no_identificado'] = False
    df_copy['similar_a_fila'] = None
    df_copy['campo_similar'] = None
    df_copy['porcentaje_similitud'] = None
    df_copy['contenido_similar'] = None
    
    pares_similares = 0
    comparaciones_totales = 0
    
    for seccion, grupo_seccion in df_copy.groupby('seccion'):
        print(f"\nProcesando sección: {seccion}")
        
        for cell, grupo_cell in grupo_seccion.groupby('cell'):
            print(f"  Célula: {cell}")
            
            no_identificadas = grupo_cell[grupo_cell['numero_tabla_grupo'] == 'no identificado']
            identificadas = grupo_cell[grupo_cell['numero_tabla_grupo'] != 'no identificado']
            
            if len(no_identificadas) == 0 or len(identificadas) == 0:
                continue

            # 1. Recolectar todos los textos únicos
            textos_a_procesar = set()
            for _, row in pd.concat([no_identificadas, identificadas]).iterrows():
                if pd.notna(row.get('justificacion')):
                    textos_a_procesar.add(row['justificacion'])
                if pd.notna(row.get('resumen')):
                    textos_a_procesar.add(row['resumen'])

            # 2. Obtener todos los embeddings en una sola llamada
            if not textos_a_procesar:
                continue
            
            print(f"    Obteniendo embeddings para {len(textos_a_procesar)} textos únicos en esta célula...")
            embeddings_cache = get_embeddings_batch(list(textos_a_procesar), client, deployment_name)
            
            if not embeddings_cache:
                print("    No se pudieron obtener los embeddings para esta célula.")
                continue

            # 3. Comparar localmente usando los embeddings cacheados
            for idx_no_id, fila_no_id in no_identificadas.iterrows():
                mejor_similitud = 0
                mejor_idx_id = None
                mejor_campo = None

                for idx_id, fila_id in identificadas.iterrows():
                    comparaciones_totales += 1
                    
                    # Comparar justificación
                    texto1_just = fila_no_id.get('justificacion')
                    texto2_just = fila_id.get('justificacion')
                    emb1 = embeddings_cache.get(texto1_just)
                    emb2 = embeddings_cache.get(texto2_just)

                    if emb1 is not None and emb2 is not None:
                        cos_sim = dot(emb1, emb2) / (norm(emb1) * norm(emb2))
                        cos_sim = max(min(cos_sim, 1.0), 0.0) * 100
                        if cos_sim > mejor_similitud:
                            mejor_similitud = cos_sim
                            mejor_idx_id = idx_id
                            mejor_campo = "justificacion"
                    
                    # Comparar resumen
                    texto1_res = fila_no_id.get('resumen')
                    texto2_res = fila_id.get('resumen')
                    emb1 = embeddings_cache.get(texto1_res)
                    emb2 = embeddings_cache.get(texto2_res)

                    if emb1 is not None and emb2 is not None:
                        cos_sim = dot(emb1, emb2) / (norm(emb1) * norm(emb2))
                        cos_sim = max(min(cos_sim, 1.0), 0.0) * 100
                        if cos_sim > mejor_similitud:
                            mejor_similitud = cos_sim
                            mejor_idx_id = idx_id
                            mejor_campo = "resumen"

                # Marcar si se encontró una similitud superior al umbral
                if mejor_similitud > umbral_similitud and mejor_idx_id is not None:
                    pares_similares += 1
                    df_copy.loc[idx_no_id, 'es_duplicado_no_identificado'] = True
                    df_copy.loc[idx_no_id, 'similar_a_fila'] = mejor_idx_id
                    df_copy.loc[idx_no_id, 'campo_similar'] = mejor_campo
                    df_copy.loc[idx_no_id, 'porcentaje_similitud'] = mejor_similitud
                    df_copy.loc[idx_no_id, 'contenido_similar'] = fila_no_id.get(mejor_campo, '')
            
    return df_copy, {
        'pares_similares': pares_similares,
        'comparaciones': comparaciones_totales,
        'filas_marcadas': df_copy['es_duplicado_no_identificado'].sum(),
        'llamadas_api': embedding_calls,
        'tokens_totales': total_embedding_tokens,
        'costo_total': total_embedding_cost
    }


def eliminar_duplicados_marcados(df):
    """
    Elimina las filas que fueron marcadas como duplicados según la columna 'es_duplicado_no_identificado'
    """
    filas_marcadas = df['es_duplicado_no_identificado'].sum()
    df_limpio = df[~df['es_duplicado_no_identificado']]
    
    print(f"Se eliminaron {filas_marcadas} filas marcadas como duplicados.")
    print(f"DataFrame original: {len(df)} filas")
    print(f"DataFrame limpio: {len(df_limpio)} filas")
    
    return df_limpio


def procesar_similitudes_y_marcar(df, client, deployment_name, umbral_similitud=75):
    """
    Función completa que integra la detección de similitud por embeddings y 
    marca las filas para su posterior eliminación.
    """
    print("\n=== FASE 1: DETECCIÓN Y MARCADO DE 'NO IDENTIFICADOS' SIMILARES ===")
    df_marcado, stats_fase1 = marcar_similitudes_no_identificadas(
        df, 
        client,
        deployment_name,
        umbral_similitud=umbral_similitud
    )
    
    print("\n====== RESUMEN DEL PROCESO DE MARCADO ======")
    print(f"Filas en el DataFrame original: {len(df)}")
    print(f"Filas marcadas como duplicados: {df_marcado['es_duplicado_no_identificado'].sum()}")
    
    print("\nUso de embeddings:")
    print(f"Total de llamadas a la API: {embedding_calls}")
    print(f"Total de tokens procesados: {total_embedding_tokens}")
    print(f"Costo total estimado: ${total_embedding_cost:.6f} USD")
    print("=================================================")
    

    print("\nPara eliminar las filas marcadas, puede llamar a la función 'eliminar_duplicados_marcados(df_marcado)'")
    
    return df_marcado


def count_tokens_embeddings(text, model="text-embedding-3-large"):
    """Cuenta los tokens para el modelo de embeddings especificado."""
    if "text-embedding-3" in model:
        encoding = tiktoken.get_encoding("cl100k_base")
    else: 
        encoding = tiktoken.get_encoding("p50k_base")
    token_list = encoding.encode(text)
    return len(token_list)


In [ ]:
embedding_costs = {"text-embedding-3-large": 0.00013}
total_embedding_tokens = 0
total_embedding_cost = 0.0
embedding_calls = 0

df_marcado = procesar_similitudes_y_marcar(
    df_sin_duplicados,
    client, 
    deployment_name="text-embedding-3-small",
    umbral_similitud=75
)

In [ ]:
df_limpio = eliminar_duplicados_marcados(df_marcado)
print(f"se eliminó un {(len(df_marcado) - len(df_limpio))*100/len(df_marcado)} % de los datos")
df_limpio.to_excel(os.path.join(path, "obligaciones_final_final_2.xlsx"), index=False)

### Contingencias y Emergencias Categorizadas

In [ ]:
df = df_limpio.copy()

In [ ]:
class TipologiaPrincipalCE(str, Enum):
    RIESGOS_OPERACIONALES = "Riesgos Operacionales"
    RIESGOS_NATURALES = "Riesgos Naturales"  
    RIESGOS_ANTRÓPICOS = "Riesgos Antrópicos"
    NO_CLASIFICABLE = "No Clasificable"

class SubTipologiaCE(str, Enum):
    # Para Riesgos Operacionales
    FALLA_DE_EQUIPOS_E_INFRAESTRUCTURA = "Falla de Equipos e Infraestructura"
    DERRAME_O_FUGA_DE_PROCESO = "Manejo de Sustancias y Residuos Peligrosos"
    ACCIDENTE_LABORAL_O_DE_TRANSPORTE = "Accidente Laboral o de Transporte"
    INCENDIO_O_EXPLOSION_DE_ORIGEN_TECNICO = "Incendio o Explosión de Origen Técnico"
    
    # Para Riesgos Naturales
    INCENDIOS = "Incendios de Origen No Operacional"
    ESTABILIDAD_GEOTECNICA = "Eventos Geofísicos y Geotécnicos"
    EVENTO_HIDROMETEOROLOGICO = "Eventos Hidrometeorológicos"
    CONTAMINACION = "Contaminación y Afectación del Aire, Agua y Suelo"
    AFECTACION_A_LA_BIOTA = "Afectación a la Biota"

    # Para Riesgos Antrópicos
    CONFLICTO_SOCIAL_O_COMUNITARIO = "Relacionamiento Comunitario y Social"
    PATRIMONIO_CULTURAL = "Patrimonio Cultural"
    ACTO_ILICITO_O_VANDALICO = "Actos Ilícitos"
    EMERGENCIA_SANITARIA = "Emergencias Sanitarias y Gestión de Residuos"
    
    # Para No Clasificable
    DESCRIPCION_GENERAL_O_DOCUMENTAL = "Descripción General o Documental"
    OTRO = "Otro"

class ClasificacionContingencia(BaseModel):
    tipologia_principal: TipologiaPrincipalCE = Field(description="Categoría principal de la contingencia o emergencia.")
    sub_tipologia: SubTipologiaCE = Field(description="Subcategoría específica dentro de la tipología principal.")

MAPEO_TIPOLOGIAS = {
    TipologiaPrincipalCE.RIESGOS_OPERACIONALES: [
        SubTipologiaCE.FALLA_DE_EQUIPOS_E_INFRAESTRUCTURA,
        SubTipologiaCE.DERRAME_O_FUGA_DE_PROCESO,
        SubTipologiaCE.ACCIDENTE_LABORAL_O_DE_TRANSPORTE,
        SubTipologiaCE.INCENDIO_O_EXPLOSION_DE_ORIGEN_TECNICO,
    ],
    TipologiaPrincipalCE.RIESGOS_NATURALES: [
        SubTipologiaCE.INCENDIOS,
        SubTipologiaCE.ESTABILIDAD_GEOTECNICA,
        SubTipologiaCE.EVENTO_HIDROMETEOROLOGICO,
        SubTipologiaCE.CONTAMINACION,
        SubTipologiaCE.AFECTACION_A_LA_BIOTA,
    ],
    TipologiaPrincipalCE.RIESGOS_ANTRÓPICOS: [
        SubTipologiaCE.CONFLICTO_SOCIAL_O_COMUNITARIO,
        SubTipologiaCE.PATRIMONIO_CULTURAL,
        SubTipologiaCE.ACTO_ILICITO_O_VANDALICO,
        SubTipologiaCE.EMERGENCIA_SANITARIA,
    ],
    TipologiaPrincipalCE.NO_CLASIFICABLE: [
        SubTipologiaCE.DESCRIPCION_GENERAL_O_DOCUMENTAL,
        SubTipologiaCE.OTRO,
    ]
}

class ClasificacionContingencia(BaseModel):
    tipologia_principal: TipologiaPrincipalCE = Field(description="Categoría principal de la contingencia o emergencia.")
    sub_tipologia: SubTipologiaCE = Field(description="Subcategoría específica dentro de la tipología principal.")

    @model_validator(mode='after')
    def validar_subtipologia_segun_tipologia(self) -> 'ClasificacionContingencia':
        tipologia = self.tipologia_principal
        sub_tipologia = self.sub_tipologia

        if sub_tipologia not in MAPEO_TIPOLOGIAS.get(tipologia, []):
            raise ValueError(f"La sub-tipología '{sub_tipologia.value}' no es válida para la tipología principal '{tipologia.value}'.")

        return self

try:
    clasificacion_valida = ClasificacionContingencia(
        tipologia_principal="Riesgos Naturales",
        sub_tipologia="Eventos Hidrometeorológicos"
    )
    print("Validación exitosa:")
    print(clasificacion_valida.model_dump_json(indent=2))
except ValidationError as e:
    print(e)

print("\n" + "="*40 + "\n")

try:
    clasificacion_invalida = ClasificacionContingencia(
        tipologia_principal="Riesgos Naturales",
        sub_tipologia="Falla de Equipos e Infraestructura"  
    )
except ValidationError as e:
    print("Validación fallida como se esperaba:")
    print(e)

system_instructions = """
                        Eres un experto en gestión de riesgos y planes de contingencia para proyectos de gran escala (minería, energía, infraestructura). Tu tarea es clasificar descripciones de contingencias o emergencias o acciones según una tipología específica y estructurada.
                        
                        **CATEGORÍAS Y DEFINICIONES:**
                        
                        **1. Riesgos Operacionales:** Fallas, errores o accidentes originados DENTRO de la operación del proyecto. La causa es técnica o humana no intencional.
                           - **Falla de Equipos e Infraestructura:** Una máquina se rompe, una tubería se corroe, un sistema eléctrico falla.
                           - **Manejo de Sustancias y Residuos Peligrosos:** Un derrame o fuga causado por una falla en un proceso, una bomba, estanque, relave u otro.
                           - **Accidente Laboral o de Transporte:** Una persona se accidenta operando maquinaria; un camión del proyecto choca por error del conductor.
                           - **Incendio o Explosión de Origen Técnico:** El fuego o explosión, comienza por un cortocircuito, sobrecalentamiento, tronadura, falla de un equipo, acumulación de gases u otro.
                        
                        **2. Riesgos Naturales:** Eventos externos gatillados por el entorno físico o cuyo impacto principal es ambiental sin una causa operacional.
                           - **Incendios de Origen No Operacional:**  Fuego en vegetación circundante que afecta o es afectado por el proyecto.
                           - **Eventos Geofísicos y Geotécnicos:** Sismos, erupciones, aluviones, o la inestabilidad de un botadero, talud o tranque.
                           - **Eventos Hidrometeorológicos:** Lluvias torrenciales, inundaciones, nevazones, sequías, tormentas eléctricas.
                           - **Contaminación y Afectación del Aire, Agua y Suelo:**  Emisiones; degradación de suelos;  alteración de cauces, afloramientos y contaminación en general.
                           - **Afectación a la Biota:** El texto describe principalmente un daño a animales o vegetación (ej. atropello de fauna).
                        
                        **3. Riesgos Antrópicos:** Acciones o conflictos asociados a personas o grupos humanos.
                           - **Relacionamiento Comunitario y Social:** Protestas, bloqueos, huelgas, conflictos con comunidades.
                           - **Patrimonio Cultural:** Hallazgo o afectación de sitios arqueológicos o paleontológicos.
                           - **Actos Ilícitos:** Robo, hurto, sabotaje, vandalismo, ataques intencionados.
                           - **Emergencias Sanitarias y Gestión de Residuos:** Proliferación de enfermedades, problemas sanitarios por mala gestión de residuos, entre otros.
                        
                        **4. No Clasificable:**
                           - **Descripción General o Documental:** El texto no describe un evento de riesgo, sino un plan, un protocolo, un procedimiento, o es excesivamente vago ("Todo el proyecto", "Control y seguimiento").
                        
                        **INSTRUCCIONES CLAVE:**
                        1.  Lee atentamente la descripción de la contingencia.
                        2.  Clasifica en la **tipología principal** la que sea MÁS representativa. 
                        3.  Selecciona la **subtipología** que MEJOR se ajuste DENTRO de esa categoría principal.
                        4.  Entiende las categorías en un sentido amplio (es posible que no cuadre exactamente, sé flexible). 
                        5.  **USA "NO CLASIFICABLE" SÓLO COMO ÚLTIMO RECURSO:** Aplícalo solo para frases vagas, u otras acciones de control que sean extremadamente difíciles de clasificar.
                        5.  Responde ÚNICAMENTE en el formato JSON solicitado. No agregues explicaciones ni texto adicional fuera del JSON.
                        """

initial_messages = [
    {
        "role": "user",
        "content": """Acciones de contingencia para el área de espesado de relaves y relaveducto. Derrame de pulpa de relaves y líquidos en área de espesado. - Derrames de relaves desde relaveducto. - Derrames de agua de procesos en el trazado de cañerías de recirculación de agua."""
    },
    {
        "role": "assistant",
        "content": """
        {
            "tipologia_principal": "Riesgos Operacionales",
            "sub_tipologia": "Manejo de Sustancias y Residuos Peligrosos"
        }
        """
    },
    {
        "role": "user",
        "content": """Acciones ante contingencias en caminos de acceso durante la operación. Contingencias relacionadas con caminos de acceso en donde se puedan producir contingencias tal como deslizamientos tierras y rocas."""
    },
    {
        "role": "assistant",
        "content": """
        {
            "tipologia_principal": "Riesgos Naturales",
            "sub_tipologia": "Eventos Geofísicos y Geotécnicos"
        }
        """
    },
    {
        "role": "user",
        "content": """Acciones de emergencia para oficinas y talleres. Posibilidad que se generen incendios y explosiones por una mala ejecución de protocolos en cuanto al manejo de materiales inflamables y/o combustibles, así como también la ocurrencia de una contingencia eléctrica."""
    },
    {
        "role": "assistant",
        "content": """
        {
            "tipologia_principal": "Riesgos Operacionales",
            "sub_tipologia": "Incendio o Explosión de Origen Técnico"
        }
        """
    },
    {
        "role": "user",
        "content": """Acciones de contingencia para piscinas y plataformas en fase de construcción. Riesgo de alteración accidental de sitios arqueológicos"""
    },
    {
        "role": "assistant",
        "content": """
        {
            "tipologia_principal": "Riesgos Antrópicos",
            "sub_tipologia": "Patrimonio Cultural"
        }
        """
    },
    {
        "role": "user",
        "content": """Acciones de contingencia ante procesos erosivos y riesgos a la seguridad de los trabajadores"""
    },
    {
        "role": "assistant",
        "content": """
        {
            "tipologia_principal": "Riesgos Operacionales",
            "sub_tipologia": "Accidente Laboral o de Transporte"
        }
        """
    },
    {
        "role": "user",
        "content": """Acciones de contingencia para el Sistema de Transporte de Relaves"""
    },
    {
        "role": "assistant",
        "content": """
        {
            "tipologia_principal": "Riesgos Operacionales",
            "sub_tipologia": "Falla de Equipos e Infraestructura"
        }
        """
    }
]

In [ ]:

def clasificar_iniciativa(client, texto_a_clasificar):
    """
    Clasifica un texto dado utilizando un modelo de IA con instrucciones de sistema
    y ejemplos de few-shot para guiar la respuesta.
    """
    messages = copy.deepcopy(initial_messages)
    messages.insert(0, {"role": "system", "content": system_instructions})
    messages.append({"role": "user", "content": texto_a_clasificar})
    
    try:

        completion = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            temperature=0,
            response_format=ClasificacionContingencia,
            messages=messages,
            timeout=30.0,  
        )
        
        
        resultado = completion.choices[0].message.parsed
        return resultado.tipologia_principal.value, resultado.sub_tipologia.value
        
    except Exception as e:
        print(f"Error al clasificar iniciativa: {e}")
        return "Error en API", "Error en API"

def clasificar_iniciativa(client, texto_a_clasificar, max_retries=2):
    """
    Clasifica un texto, reintentando con retroalimentación si ocurre un error de validación.
    """
    messages = copy.deepcopy(initial_messages)
    messages.insert(0, {"role": "system", "content": system_instructions})
    messages.append({"role": "user", "content": texto_a_clasificar})

    # Bucle para controlar los reintentos
    for attempt in range(max_retries):
        try:
            completion = client.beta.chat.completions.parse(
                model="gpt-4o-mini",
                temperature=0,
                response_format=ClasificacionContingencia,
                messages=messages,
                timeout=30.0,
            )
            
            resultado = completion.choices[0].message.parsed
            return resultado.tipologia_principal.value, resultado.sub_tipologia.value

        except ValidationError as e:
            print(f"🚨 Error de Validación en el intento {attempt + 1}:")
            print(e)

            if attempt == max_retries - 1:
                print("❌ Se alcanzó el número máximo de reintentos. La clasificación falló.")
                break 

            
            feedback_message = (
                "Tu respuesta anterior no fue válida. "
                f"Error: {e}. "
                "Por favor, corrige tu respuesta para que la 'sub_tipologia' sea una opción válida "
                "para la 'tipologia_principal' que seleccionaste. Revisa las reglas y genera una nueva respuesta JSON válida."
            )
            
            messages.append({"role": "user", "content": feedback_message})
            
        except Exception as e:
            print(f"--- Error General de API ---")
            print(f"Ocurrió un error inesperado: {e}")
            return "Error en API", "La solicitud falló"

    return "Clasificación Fallida", "El modelo no pudo generar una respuesta válida tras varios intentos."

In [ ]:
def main():
    try:
        df["Tipologia_CE"] = ""
        df["Subtipologia_CE"] = ""

        filas_a_clasificar = df[
            (df['seccion'] == 'contingencias_emergencias') & 
            (df['resumen'].str.len() > 5)
        ]
        
        if filas_a_clasificar.empty:
            print("No se encontraron filas que cumplan con las condiciones. Finalizando.")
            return

        print(f"DataFrame total: {len(df)} filas.")
        print(f"Se clasificarán {len(filas_a_clasificar)} filas que cumplen con los criterios. 🤖")
        
        ruta_salida = os.path.join(path, "obligaciones_clasificadas.xlsx")
        for i, (index, row) in enumerate(tqdm(filas_a_clasificar.iterrows(), total=len(filas_a_clasificar), desc="Clasificando")):
            try:
                texto_a_clasificar = f"{row['resumen']} {row['justificacion']}"
                
                tipologia, subtipologia = clasificar_iniciativa(
                    client,
                    texto_a_clasificar
                )
                
                df.at[index, "Tipologia_CE"] = tipologia
                df.at[index, "Subtipologia_CE"] = subtipologia
                if (i + 1) % 3000 == 0:
                    print(f"\nGuardando progreso en la fila {i+1}...")
                    df.to_excel(ruta_salida, index=False)
                
                time.sleep(0.4)
                
            except Exception as e:
                print(f"Error procesando fila con índice {index}: {e}")
                df.at[index, "Tipologia_CE"] = "Error en Proceso"
                df.at[index, "Subtipologia_CE"] = "Error en Proceso"

        print("\n✅ Proceso de clasificación completado.")
        
        # --- GUARDADO FINAL ---
        print(f"Guardando resultados finales en: {ruta_salida}")
        df.to_excel(ruta_salida, index=False)
        
    except FileNotFoundError:
        print(f"❌ Error: No se pudo encontrar el archivo de entrada.")
    except KeyError:
        print("❌ Error: Asegúrate de que las columnas 'seccion' y 'resumen' existan.")
    except Exception as e:
        print(f"❌ Ocurrió un error general: {e}")

if __name__ == "__main__":
    main()

In [ ]:
df.to_excel(os.path.join(path, "obligaciones_clasificadas.xlsx"))

### Obligaciones Expandidas

In [ ]:
path = os.path.join(os.environ['USERPROFILE'], "RUTA")
df0 = pd.read_excel(os.path.join(path, "obligaciones_clasificadas.xlsx"))

#### Eliminar más duplicados

In [ ]:
df_procesado = df0.copy()

# --- CÁLCULO ESTADÍSTICO ---
total_inicial = len(df_procesado)
pas_inicial = len(df_procesado[df_procesado['seccion'] == 'pas'])
contingencias_inicial = len(df_procesado[df_procesado['seccion'] == 'contingencias_emergencias'])

# --- INICIO: PROCESAMIENTO DEL DATAFRAME ---
df_a_mantener = df_procesado[~df_procesado['seccion'].isin(['pas', 'contingencias_emergencias'])].copy()
df_a_procesar = df_procesado[df_procesado['seccion'].isin(['pas', 'contingencias_emergencias'])].copy()
df_a_procesar['largo_contenido'] = df_a_procesar['resumen'].str.len().fillna(0) + \
                                   df_a_procesar['justificacion'].str.len().fillna(0)

df_a_procesar = df_a_procesar.sort_values('largo_contenido', ascending=False)
columnas_clave = ['cell', 'seccion', 'numero_tabla']
df_sin_duplicados = df_a_procesar.drop_duplicates(subset=columnas_clave, keep='first')
df_sin_duplicados = df_sin_duplicados.drop(columns=['largo_contenido'])


df_final = pd.concat([df_sin_duplicados, df_a_mantener], ignore_index=True)
# --- FIN: PROCESAMIENTO DEL DATAFRAME ---


# --- INICIO: IMPRESIÓN DE RESULTADOS ---
total_final = len(df_final)
pas_final = len(df_final[df_final['seccion'] == 'pas'])
contingencias_final = len(df_final[df_final['seccion'] == 'contingencias_emergencias'])

filas_eliminadas = total_inicial - total_final
pas_eliminadas = pas_inicial - pas_final
contingencias_eliminadas = contingencias_inicial - contingencias_final


porcentaje_total_eliminado = (filas_eliminadas / total_inicial) * 100 if total_inicial > 0 else 0
porcentaje_pas_eliminado = (pas_eliminadas / pas_inicial) * 100 if pas_inicial > 0 else 0
porcentaje_contingencias_eliminado = (contingencias_eliminadas / contingencias_inicial) * 100 if contingencias_inicial > 0 else 0


print("\n" + "="*60 + "\n")
print("📊 --- Estadísticas de Eliminación --- 📊")
print(f"Filas eliminadas: {filas_eliminadas} de {total_inicial} ({porcentaje_total_eliminado:.2f}% del total).")
print(f"Filas de 'pas' eliminadas: {pas_eliminadas} de {pas_inicial} ({porcentaje_pas_eliminado:.2f}% de las filas 'pas').")
print(f"Filas de 'contingencias_emergencias' eliminadas: {contingencias_eliminadas} de {contingencias_inicial} ({porcentaje_contingencias_eliminado:.2f}% de las filas 'contingencias').")
print("\n" + "="*60 + "\n")

In [ ]:
# Condición para seleccionar las filas
df = df_final.copy()
condicion = (df['seccion'] == 'compromisos_voluntarios') & (df['fuente'] == "['decisión de autoridad']")
df.loc[condicion, 'fuente'] = "['Compromiso ambiental voluntario']"
condicion = (df['fuente'] == "[]") | (df['numero_fuente'] == "[]") 
df.loc[condicion, 'fuente'] = "['decisión de autoridad']"
df.loc[condicion, 'numero_fuente'] = "['']"

def clean_numero_fuente(numero_fuente_str):
    try:
        lista = ast.literal_eval(numero_fuente_str)
        if len(lista) == 1 and isinstance(lista[0], str):
            if ',' in lista[0]:
                parts = [part.strip() for part in lista[0].split(',')]
                return str(parts)
            elif 'y' in lista[0]:
                parts = [part.strip() for part in lista[0].split('y')]
                return str(parts)
        return str(lista)
    except (ValueError, SyntaxError):
        return numero_fuente_str

df['numero_fuente'] = df['numero_fuente'].apply(clean_numero_fuente)

#### Expandir

In [ ]:

def expandir_fuentes_modificado(df):
    """
    Expande las filas del DataFrame basado en las columnas 'fuente' y 'numero_fuente',
    identifica las filas no procesadas y cuenta las obligaciones.
    
    Devuelve:
        - pd.DataFrame: El nuevo DataFrame expandido (df2).
        - list: Una lista con los índices de las filas del df original que no se incluyeron.
        - dict: Un diccionario con el recuento de obligaciones en el df original y en el nuevo.
    """
    filas_expandidas = []
    filas_no_incluidas_indices = []
    

    for idx, row in df.iterrows():
        try:
            fuentes = ast.literal_eval(row["fuente"])
            numeros_fuentes = ast.literal_eval(row["numero_fuente"])
            
            if not isinstance(fuentes, list) or not isinstance(numeros_fuentes, list):
                raise ValueError("El contenido no es una lista")

        except (ValueError, SyntaxError):
            filas_no_incluidas_indices.append(idx)
            continue
        
        if not fuentes or not numeros_fuentes:
            filas_no_incluidas_indices.append(idx)
            continue
        
        obligacion_id = str(uuid.uuid4())
        
        if len(fuentes) > 1:
            for fuente, numero in zip(fuentes, numeros_fuentes):
                nueva_fila = row.copy()
                nueva_fila["obligacion_id"] = obligacion_id
                nueva_fila["fuente"] = str(fuente).replace("['", "").replace("']", "")
                nueva_fila["numero_fuente"] = str(numero).replace("['", "").replace("']", "")
                filas_expandidas.append(nueva_fila)
        else:
            row["obligacion_id"] = obligacion_id
            row["fuente"] = str(fuentes[0]).replace("['", "").replace("']", "")
            row["numero_fuente"] = str(numeros_fuentes[0]).replace("['", "").replace("']", "")
            filas_expandidas.append(row)
    
    df_expandido = pd.DataFrame(filas_expandidas).reset_index(drop=True)
    
    recuentos = {
        "Obligaciones en df original": len(df),
        "Obligaciones procesadas en df2": df_expandido["obligacion_id"].nunique() if not df_expandido.empty else 0
    }
    
    return df_expandido, filas_no_incluidas_indices, recuentos

df2, filas_omitidas, conteos = expandir_fuentes_modificado(df)

print("📊 RECUENTO DE OBLIGACIONES:")
for nombre, valor in conteos.items():
    print(f"- {nombre}: {valor}")
print("-" * 30)

if filas_omitidas:
    print(f"⚠️ Se omitieron {len(filas_omitidas)} fila(s) del DataFrame original debido a errores de formato o datos vacíos.")
    print("Filas no incluidas (índices: {}):".format(', '.join(map(str, filas_omitidas))))
    print(df.loc[filas_omitidas])
else:
    print("👍 Todas las filas del DataFrame original fueron procesadas exitosamente.")

print("-" * 30)
print("🔍 VISTA PREVIA DEL NUEVO DATAFRAME (df2):")
print(df2.head())

#### Limpieza

In [ ]:
def limpiar_y_estandarizar_fuentes(row):
    fuente = row['fuente']
    numero_fuente = str(row['numero_fuente'])

    reglas = [
        (r'\b(NCh\s*Elec)\b', 'norma chilena electrica'),
        (r'\b(Norma\s*Chilena|NCH|NCh)\b', 'norma chilena'),
        (r'\b(DFL)\b', 'decreto con fuerza de ley'),
        (r'\b(DS)\b', 'decreto supremo'),
        (r'\b(Ley)\b', 'ley'),
        (r'\b(ordenanza general|ordenanza municipal|ordenanza|ORD)\b', 'ordenanza'),
        (r'\b(Constitución\s*Política\s*de\s*la\s*República|CPR|constitucion)\b', 'constitucion politica'),
        (r'\b(Resolución\s*Exenta|Resolución|R.E.|RES EX|RES N°|RES Nº|RES|RE)\b', 'resolucion'),
        (r'\b(Ordenanza\s*General\s*de\s*Urbanismo\s*y\s*Construcciones|OGUC)\b', 'oguc'),
        (r'\b(Plan\s*Regulador)\b', 'plan regulador'),
        (r'\b(ordenanza\s*municipal)\b', 'ordenanza municipal'),
        (r'\b(RGR)\b', 'rgr'),
        (r'\b(NSEG)\b', 'nseg'),
        (r'\b(NSEC)\b', 'nsec')
    ]

    for patron, nueva_fuente in reglas:
        if re.search(patron, numero_fuente, re.IGNORECASE):
            print(rf"Encontrado patrón {patron} en {numero_fuente}")
            fuente = nueva_fuente
            numero_fuente = re.sub(patron, '', numero_fuente, flags=re.IGNORECASE).strip()
            break
            
    patron_limpieza = r'\b(Norma\s*Oficial|Norma|No)\b|\b(N\s*°|N\s*º|N°|Nº)'
    numero_fuente = re.sub(patron_limpieza, '', numero_fuente, flags=re.IGNORECASE)
    numero_fuente = re.sub(r'^N(\d)', r'\1', numero_fuente.strip())
    numero_fuente = numero_fuente.replace(':', '/')
    numero_fuente = numero_fuente.replace('.', '')
    numero_fuente = numero_fuente.replace(' Of ', '/')
    numero_fuente = numero_fuente.replace(' of ', '/')
    numero_fuente = numero_fuente.replace('Of', '/')
    numero_fuente = numero_fuente.replace('of', '/')
    numero_fuente = ' '.join(numero_fuente.split())

    return pd.Series([fuente, numero_fuente])

df2[['fuente', 'numero_fuente']] = df2.apply(limpiar_y_estandarizar_fuentes, axis=1)

#### Normalizar Año Fuente

In [ ]:
def normalizar_ano_fuente_corregido(valor):
    """
    Normaliza y valida un valor de fuente.
    - Convierte formatos como 'número/aa' o números enteros largos a 'código/aaaa'.
    - Valida que el año resultante esté entre 1900 y 2025.
    - Si el año es inválido, imprime una advertencia y devuelve el valor original.
    """
    if valor is None:
        return None
    valor_str = str(valor).strip()
    
    codigo_final = None
    ano_final_str = None

    m = re.match(r'^(\d+)/(\d{2})$', valor_str)
    if m:
        codigo_final, ano_corto = m.groups()
        ano_corto_int = int(ano_corto)
        if 0 <= ano_corto_int <= 25:
            ano_final_str = f"20{ano_corto.zfill(2)}"
        else:
            ano_final_str = f"19{ano_corto.zfill(2)}"

    elif valor_str.isdigit():
        longitud = len(valor_str)
        # Ejemplo: 12342023 -> 1234/2023
        if longitud == 8:
            codigo_final = valor_str[:4]
            ano_final_str = valor_str[4:]
        # Ejemplo: 1232023 -> 123/2023
        elif longitud == 7:
            codigo_final = valor_str[:3]
            ano_final_str = valor_str[3:]

    if ano_final_str:
        ano_int = int(ano_final_str)
        if 1900 <= ano_int <= 2025:
            return f"{codigo_final}/{ano_final_str}"
        else:
            print(f"⚠️ Advertencia: El año '{ano_int}' extraído de '{valor_str}' está fuera del rango (1900-2025).")
            return valor_str
            
    return valor_str

df2["numero_fuente"] = df2["numero_fuente"].apply(normalizar_ano_fuente_corregido)
print("Normalización corregida aplicada.")

In [ ]:
df2['ano_num'] = pd.to_numeric(df2['numero_fuente'], errors='coerce')
df2['group_size'] = df2.groupby('obligacion_id')['obligacion_id'].transform('size')
filas_a_procesar_mask = (
    (df2['ano_num'] >= 1900) &
    (df2['ano_num'] <= 2025) &
    (df2['group_size'] > 1) &
    (df2['ano_num'].notna())
)
mapa_obligacion_ano = df2[filas_a_procesar_mask].set_index('obligacion_id')['ano_num'].astype(int).astype(str).to_dict()

for obligacion_id, ano_a_agregar in mapa_obligacion_ano.items():
    filas_a_modificar_mask = (
        (df2['obligacion_id'] == obligacion_id) &
        (df2['numero_fuente'] != ano_a_agregar) &
        (~df2['numero_fuente'].str.contains('/'))
    )
    df2.loc[filas_a_modificar_mask, 'numero_fuente'] = df2.loc[filas_a_modificar_mask, 'numero_fuente'] + '/' + ano_a_agregar

indices_a_eliminar = df2[filas_a_procesar_mask].index
df2 = df2.drop(indices_a_eliminar)


df2 = df2.drop(columns=['ano_num', 'group_size'])

#### Definir Variables de Fuentes

In [ ]:
numero_fuente_counts = df2["numero_fuente"].value_counts()
reemplazos = {}

for fuente in numero_fuente_counts.index:
    fuente_str = str(fuente)
    if fuente_str and not re.search(r'/\d+$', fuente_str) and fuente != 1:
        posibles_coincidencias = [
            f for f in numero_fuente_counts.index 
            if re.match(rf'^{re.escape(fuente_str)}/\d+$', str(f))
        ]
        if posibles_coincidencias:
            mejor_coincidencia = max(posibles_coincidencias, key=lambda x: numero_fuente_counts[x])
            reemplazos[fuente_str] = mejor_coincidencia
            print(fuente_str, "a", mejor_coincidencia)
df2["numero_fuente"] = df2["numero_fuente"].replace(reemplazos)
df2['numero_fuente_completo'] = df2['numero_fuente']
df2['año_fuente'] = None

def extraer_ano(valor):
    if isinstance(valor, str):
        if '/' in valor:
            parts = valor.split('/')
            return parts[1]
        if '-' in valor:
            parts = valor.split('-')
            if len(parts) == 2 and parts[1].isdigit():
                return parts[1]
    return None

def extraer_numero(valor):
    if isinstance(valor, str):
        if '/' in valor:
            return valor.split('/')[0]
        if '-' in valor:
            return valor.split('-')[0]
    return valor


df2['año_fuente'] = df2['numero_fuente_completo'].apply(extraer_ano)
df2['numero_fuente'] = df2['numero_fuente_completo'].apply(extraer_numero)
df2['numero_fuente'] = df2['numero_fuente'].str.replace('.', '', regex=False)

# Convertir año_fuente a número entero cuando sea posible
def convertir_a_entero(valor):
    if valor is not None:
        try:
            return int(valor)
        except (ValueError, TypeError):
            return valor
    return valor

df2['año_fuente'] = df2['año_fuente'].apply(convertir_a_entero)

#### Limpieza Normas Identificadas

In [ ]:
lista_normas_raw = """
decreto supremo 47/1992
decreto supremo 100/2005
decreto supremo 109/2017
decreto supremo 112/2013
decreto con fuerza de ley 1122/1981
resolucion 1139/2013
decreto supremo 114/2002
decreto supremo 115/2004
decreto supremo 1150/1980
decreto supremo 1164/1974
decreto supremo 12/2021
ley 1215/1978
decreto supremo 125/2019
decreto supremo 1261/1957
decreto supremo 13/2011
decreto supremo 132/2002
decreto supremo 133/2005
decreto supremo 138/2005
decreto supremo 144/1961
decreto supremo 146/1997
decreto supremo 148/2003
decreto supremo 149/2006
decreto supremo 15/2013
decreto supremo 151/2006
resolucion 1518/2013
decreto supremo 157/2007
decreto supremo 158/1980
ley 15840/1964
decreto supremo 160/2008
resolucion 499/2006
decreto supremo 1665/2002
ley 16744/1968
decreto supremo 172/1988
ley 17288/1970
ley 17798/1972
decreto supremo 18/2001
ley 18248/1983
ley 18290/1984
ley 18378/1984
ley 18695/1988
ley 18755/1989
ley 18834/1989
ley 18892/1989
decreto con fuerza de ley 19/1984
ley 19253/1993
decreto supremo 193/1998
ley 19300/1994
decreto supremo 194/1973
ley 19473/1996
ley 19880/2003
decreto supremo 20/2013
decreto supremo 200/1993
ley 20001/2005
ley 20096/2006
ley 20283/2008
ley 20380/2009
ley 20389/2009
ley 20417/2010
ley 20443/2010
ley 20551/2011
ley 20724/2014
ley 20879/2015
ley 20920/2016
ley 20936/2017
decreto supremo 211/1991
ley 21455/2022
resolucion 223/2015
resolucion 232/2002
decreto supremo 236/1926
decreto supremo 244/2005
decreto supremo 248/2007
decreto ley 2565/1979
decreto supremo 259/1980
decreto supremo 276/1980
decreto supremo 279/1983
decreto supremo 29/2011
decreto supremo 291/2007
decreto supremo 294/1984
decreto supremo 298/1994
decreto supremo 30/2012
decreto supremo 300/1994
decreto supremo 327/1997
decreto ley 3557/1980
resolucion 359/2005
decreto supremo 37/2019
decreto supremo 38/2011
decreto supremo 40/2012
decreto supremo 400/1977
decreto supremo 405/1983
decreto supremo 41/2012
decreto supremo 4188/1955
decreto supremo 42/2011
decreto supremo 430/1991
decreto supremo 4363/1931
decreto supremo 44/2017
decreto supremo 446/2006
decreto con fuerza de ley 458/1975
decreto supremo 46/2002
decreto supremo 4601/1929
decreto supremo 461/1995
decreto supremo 48/2016
decreto supremo 484/1990
decreto supremo 5/1998
decreto supremo 531/1967
decreto supremo 54/1994
decreto supremo 55/1994
ley 58/2004
decreto supremo 59/1998
decreto supremo 594/1999
decreto supremo 6/2009
resolucion 610/1982
decreto con fuerza de ley 4/2006
decreto supremo 65/2015
decreto supremo 655/1940
decreto supremo 66/2009
decreto supremo 68/2021
decreto supremo 7/2009
decreto ley 701/1974
decreto supremo 72/1985
decreto con fuerza de ley 725/1967
decreto supremo 735/1969
decreto supremo 75/1987
decreto supremo 78/2009
decreto supremo 80/2004
decreto supremo 82/2010
decreto supremo 83/2007
decreto ley 830/1974
decreto con fuerza de ley 850/1997
decreto supremo 867/1978
decreto supremo 878/2011
decreto supremo 88/2020
decreto supremo 90/2000
decreto supremo 93/2008
decreto supremo 938/2011
decreto supremo 95/2001
"""

data_corregida = []
for line in io.StringIO(lista_normas_raw).readlines():
    if not line.strip():
        continue
    fuente, numero_año = line.strip().rsplit(' ', 1)
    numero, año = numero_año.split('/')
    data_corregida.append({
        'numero_fuente_ajustado': str(numero),
        'fuente': str(fuente),
        'año_fuente': str(año)
    })

df_mapeo = pd.DataFrame(data_corregida)
df_mapeo_indexed = df_mapeo.set_index('numero_fuente_ajustado')

df2['numero_fuente'] = df2['numero_fuente'].astype(str)
df2['año_fuente'] = df2['año_fuente'].astype(str)
df2['fuente'] = df2['fuente'].astype(str)
df2['numero_fuente_ajustado'] = df2['numero_fuente']


for index, regla in df_mapeo.iterrows():
    numero_correcto = regla['numero_fuente_ajustado']
    fuente_correcta = regla['fuente']
    ano_correcto = int(regla['año_fuente'])
    mascara_filas_a_evaluar = (df2['numero_fuente_ajustado'] == numero_correcto)

    if not mascara_filas_a_evaluar.any():
        continue

    def aplicar_reglas_de_reemplazo(ano_actual_str):
        try:
            ano_actual_num = int(float(ano_actual_str))
        except (ValueError, TypeError):
            return str(ano_correcto)

        if not (1900 <= ano_actual_num <= 2025):
            return str(ano_correcto)

        if abs(ano_actual_num - ano_correcto) == 1:
            return str(ano_correcto)

        return str(ano_actual_num)

    df2.loc[mascara_filas_a_evaluar, 'año_fuente'] = df2.loc[mascara_filas_a_evaluar, 'año_fuente'].apply(aplicar_reglas_de_reemplazo)
    df2.loc[mascara_filas_a_evaluar, 'fuente'] = fuente_correcta


In [ ]:
lista_normas_maestra = """
fuente;numero_fuente_ajustado;año_fuente
decreto con fuerza de ley;1;1990
resolucion;1;1995
decreto con fuerza de ley;1;2002
decreto con fuerza de ley;1;2009
decreto supremo;1;2013
decreto con fuerza de ley;1;1982
decreto supremo;4;1994
decreto con fuerza de ley;4;20018
decreto supremo;4;2009
decreto supremo;8;1993
decreto supremo;8;2019
decreto supremo;31;2012
decreto supremo;31;2016
decreto supremo;43;2012
decreto supremo;43;2015
"""

df_master = pd.read_csv(io.StringIO(lista_normas_maestra), sep=';')
print("\n" + "="*40 + "\n")
df_to_correct = df2.copy()
df_original = df_to_correct.copy()

def clean_dataframe(df):
    df['fuente'] = df['fuente'].astype(str)
    df['numero_fuente_ajustado'] = df['numero_fuente_ajustado'].astype(str)
    df['año_fuente'] = df['año_fuente'].astype(str).str.extract(r'(\d{4})', expand=False)
    df['numero_fuente_ajustado'] = pd.to_numeric(df['numero_fuente_ajustado'], errors='coerce')
    df['año_fuente'] = pd.to_numeric(df['año_fuente'], errors='coerce')
    df['numero_fuente_ajustado'] = df['numero_fuente_ajustado'].astype('Int64')
    df['año_fuente'] = df['año_fuente'].astype('Int64')
    return df

df_master = clean_dataframe(df_master)
df_to_correct = clean_dataframe(df_to_correct)

mapa_fuente_correcta = df_master.set_index(['numero_fuente_ajustado', 'año_fuente'])['fuente'].to_dict()
df_to_correct['key'] = list(zip(df_to_correct['numero_fuente_ajustado'], df_to_correct['año_fuente']))
df_to_correct['fuente_correcta'] = df_to_correct['key'].map(mapa_fuente_correcta)
df_to_correct['fuente'] = np.where(
    (df_to_correct['fuente_correcta'].notna()) & (df_to_correct['fuente_correcta'] != df_to_correct['fuente']),
    df_to_correct['fuente_correcta'], 
    df_to_correct['fuente']          
)
df_to_correct = df_to_correct.drop(columns=['key', 'fuente_correcta'])

mapa_año_typo = {}
for _, row in df_master.iterrows():
    numero = row['numero_fuente_ajustado']
    fuente = row['fuente']
    año_correcto = row['año_fuente']
    mapa_año_typo[(numero, fuente, año_correcto - 1)] = año_correcto
    mapa_año_typo[(numero, fuente, año_correcto + 1)] = año_correcto

df_to_correct['key_typo'] = list(zip(df_to_correct['numero_fuente_ajustado'], df_to_correct['fuente'], df_to_correct['año_fuente']))
df_to_correct['año_corregido'] = df_to_correct['key_typo'].map(mapa_año_typo)
df_to_correct['año_fuente'] = np.where(
    df_to_correct['año_corregido'].notna(),
    df_to_correct['año_corregido'],
    df_to_correct['año_fuente']     
)

df_to_correct['año_fuente'] = df_to_correct['año_fuente'].astype('Int64')
df_to_correct = df_to_correct.drop(columns=['key_typo', 'año_corregido'])

### Eliminar los vacíos

In [ ]:
df_procesado = df_to_correct.copy()
mask_fuente = df_procesado['fuente'] == 'permisos ambientales sectoriales'
numeros_en_pas = pd.to_numeric(df_procesado.loc[mask_fuente, 'numero_fuente_ajustado'], errors='coerce')
mask_condiciones = (numeros_en_pas < 111) | (numeros_en_pas > 161) | (numeros_en_pas.isna())

indices_a_reemplazar = numeros_en_pas[mask_condiciones].index
df_procesado['numero_fuente_ajustado'] = df_procesado['numero_fuente_ajustado'].astype(object)
df_procesado.loc[indices_a_reemplazar, 'numero_fuente_ajustado'] = "No identificado"
df_procesado = df_procesado.drop_duplicates()

In [ ]:
df_procesado.to_excel(os.path.join(path, "df_procesasdo.xlsx"))
pas = pd.read_excel(os.path.join(path, "pas.xlsx"))

In [ ]:
df_procesado = pd.read_excel(r"RUTA\df_procesasdo.xlsx")

In [ ]:
def get_embedding(text, deployment_name):
    """Obtiene el embedding para un texto dado."""
    response = client.embeddings.create(input=[text], model=deployment_name)
    return np.array(response.data[0].embedding)

def cosine_similarity(emb1, emb2):
    """Calcula la similitud coseno entre dos embeddings."""
    return dot(emb1, emb2) / (norm(emb1) * norm(emb2)) if (norm(emb1) * norm(emb2)) != 0 else 0

print("Generando embeddings para los nombres de los PAS...")
pas_embeddings_data = []
try:
    if 'pas' in globals() and 'nombre' in pas.columns and 'numero_fuente' in pas.columns:
        for index, row in pas.iterrows():
            if pd.notna(row['nombre']):
                embedding = get_embedding(row['nombre'], "text-embedding-3-large")
                pas_embeddings_data.append((row['numero_fuente'], embedding))
        print(f"Embeddings de {len(pas_embeddings_data)} PAS generados exitosamente.")
    else:
        print("Error: El DataFrame 'pas' o las columnas 'nombre'/'numero_fuente' no están definidas.")
        pas_embeddings_data = []
except Exception as e:
    print(f"Error al generar embeddings para los PAS: {e}")
    pas_embeddings_data = []
    
mask_no_identificado = df_procesado['numero_fuente_ajustado'] == 'No identificado'
indices_a_procesar = df_procesado[mask_no_identificado].index
print(f"Se procesarán {len(indices_a_procesar)} filas con 'No identificado'.")

In [ ]:
if pas_embeddings_data:
    umbral_similitud = 0.60
    print(f"\nIniciando proceso de corrección con umbral de similitud > {umbral_similitud}...")

    for i, index in enumerate(indices_a_procesar):
        row = df_procesado.loc[index]
        
        resumen = str(row.get('resumen', ''))
        justificacion = str(row.get('justificacion', ''))
        texto_a_comparar = f"{resumen}. {justificacion}".strip()

        if not texto_a_comparar or texto_a_comparar == '.':
            print(f"Fila {i+1}/{len(indices_a_procesar)} (Índice: {index}): Omitida por falta de texto.")
            continue
            
        print("-" * 50)
        print(f"Procesando fila {i+1}/{len(indices_a_procesar)} (Índice: {index})")

        try:
            embedding_fila = get_embedding(texto_a_comparar, "text-embedding-3-large")
            similitudes_candidatos = []
            for pas_numero, pas_embedding in pas_embeddings_data:
                similitud = cosine_similarity(embedding_fila, pas_embedding)
                similitudes_candidatos.append({'pas_numero': pas_numero, 'similitud': similitud})
            

            similitudes_candidatos.sort(key=lambda x: x['similitud'], reverse=True)
            current_cell_id = row['cell']
            pas_usados_en_cell = set(
                df_procesado[
                    (df_procesado['cell'] == current_cell_id) &
                    (df_procesado.index != index) 
                ]['numero_fuente_ajustado']
            )
            
            print(f"Documento (cell): {current_cell_id}. PAS ya asignados en este doc: {pas_usados_en_cell if pas_usados_en_cell else 'Ninguno'}")

            asignacion_exitosa = False
            for candidato in similitudes_candidatos:
                pas_candidato_numero = candidato['pas_numero']
                similitud_candidato = candidato['similitud']

                if similitud_candidato > umbral_similitud:
                    if pas_candidato_numero not in pas_usados_en_cell:
                        df_procesado.loc[index, 'numero_fuente_ajustado'] = pas_candidato_numero
                        print(f"✅ Fila {index}: Corregido a PAS {pas_candidato_numero} (Similitud: {similitud_candidato:.4f}).")
                        asignacion_exitosa = True
                        break 
                    else:
                        print(f"⚠️ Candidato PAS {pas_candidato_numero} (Similitud: {similitud_candidato:.4f}) descartado. Ya está asignado en este documento.")
                else:
                    print(f"⛔ No se encontraron más candidatos por sobre el umbral de {umbral_similitud}. Máxima similitud fue {similitudes_candidatos[0]['similitud']:.4f}.")
                    break 

            if not asignacion_exitosa:
                print(f"❌ Fila {index}: No se pudo asignar un PAS. Ningún candidato cumplió los criterios de similitud y unicidad.")

        except Exception as e:
            print(f"Error procesando la fila {index}: {e}")

print("\nProceso de corrección finalizado.")

In [ ]:
df_procesado['numero_fuente_ajustado'] = df_procesado['numero_fuente_ajustado'].astype(object)
condicion_relleno = (
    (~df_procesado['fuente'].isin(['Compromiso ambiental voluntario', 'decisión de autoridad'])) &
    (df_procesado['numero_fuente_ajustado'].isnull() | (df_procesado['numero_fuente_ajustado'] == 0))
)
df_procesado.loc[condicion_relleno, 'numero_fuente_ajustado'] = df_procesado['numero_fuente_completo']
es_fuente_invalida_check = (
    df_procesado['numero_fuente_ajustado'].isnull() |
    df_procesado['numero_fuente_ajustado'].isin([0, '0', ''])
)

condicion_invalida = (
    (~df_procesado['fuente'].isin(['Compromiso ambiental voluntario', 'decisión de autoridad'])) &
    (es_fuente_invalida_check)
)


df_procesado['conteo_fuentes_validas'] = (~condicion_invalida).astype(int)
df_procesado['conteo_fuentes_validas'] = df_procesado.groupby('obligacion_id')['conteo_fuentes_validas'].transform('sum')

condicion_modificar = condicion_invalida & (df_procesado['conteo_fuentes_validas'] == 0)
df_procesado.loc[condicion_modificar, 'fuente'] = 'decisión de autoridad'

condicion_eliminar = condicion_invalida & (df_procesado['conteo_fuentes_validas'] > 0)
df_procesado = df_procesado[~condicion_eliminar].copy()

df_procesado.drop(columns=['conteo_fuentes_validas'], inplace=True)
print(f"Registros: {len(df0)}")
print(f"Registros: {len(df)}")
print(f"Registros originales: {len(df2)}, Obligaciones originales: {df2.obligacion_id.nunique()}")
print(f"Registros originales: {len(df_to_correct)}, Obligaciones originales: {df_to_correct.obligacion_id.nunique()}")
print(f"Registros procesados: {len(df_procesado)}, Obligaciones procesadas: {df_procesado.obligacion_id.nunique()}")

### Correcciones Manuales Básicas

In [ ]:
def aplicar_reglas_unificadas(df, reglas):
    """
    Aplica una serie de reglas para estandarizar las columnas de un DataFrame.

    Cada regla especifica un patrón de búsqueda y los nuevos valores para las columnas
    'fuente', 'numero_fuente_ajustado' y 'año_fuente'.
    """
    df_actualizado = df.copy()
    for regla in reglas:
        patron = regla['patron']
        fuente_nueva = regla['fuente_nueva']
        numero_nuevo = regla['numero_nuevo']
        año_nuevo = regla['año_nuevo']
        condicion = df_actualizado['numero_fuente_ajustado'].str.contains(
            patron, 
            case=False, 
            na=False, 
            regex=True
        )
        df_actualizado.loc[condicion, 'fuente'] = fuente_nueva
        df_actualizado.loc[condicion, 'numero_fuente_ajustado'] = int(numero_nuevo)
        
        if año_nuevo:
            df_actualizado.loc[condicion, 'año_fuente'] = int(año_nuevo)
            
    return df_actualizado

reglas_normativas = [
    {'patron': r'c[oó]digo\s+sanitario', 'fuente_nueva': 'código', 'numero_nuevo': '725', 'año_nuevo': '1967'},
    {'patron': r'c[oó]digo\s+de\s+aguas', 'fuente_nueva': 'código', 'numero_nuevo': '1122', 'año_nuevo': '1981'},
    {'patron': r'general\s+de\s+pesca\s+y\s+acuicultura|lgpa', 'fuente_nueva': 'ley', 'numero_nuevo': '18892', 'año_nuevo': '1989'},
    {'patron': r'ley\s+general\s+de\s+urbanismo\s+y\s+construcci[oó]n(es)?|lguc', 'fuente_nueva': 'decreto con fuerza de ley', 'numero_nuevo': '458', 'año_nuevo': '1976'},
    {'patron': r'general\s+de\s+servicios\s+el[eé]ctricos', 'fuente_nueva': 'decreto con fuerza de ley', 'numero_nuevo': '4', 'año_nuevo': '2006'},
    {'patron': r'caza', 'fuente_nueva': 'ley', 'numero_nuevo': '19473', 'año_nuevo': '1996'},
    {'patron': r'ley\s+de\s+bosques', 'fuente_nueva': 'decreto supremo', 'numero_nuevo': '4363', 'año_nuevo': '1931'},
    {'patron': r'monumentos\s+nacionales', 'fuente_nueva': 'ley', 'numero_nuevo': '17288', 'año_nuevo': '1970'},
    {'patron': r'protecci[oó]n\s+agr[ií]cola', 'fuente_nueva': 'decreto ley', 'numero_nuevo': '3557', 'año_nuevo': '1980'},
    {'patron': r'caminos', 'fuente_nueva': 'decreto con fuerza de ley', 'numero_nuevo': '850', 'año_nuevo': '1997'},
    {'patron': r'tr[aá]nsito', 'fuente_nueva': 'ley', 'numero_nuevo': '18290', 'año_nuevo': '1984'},
    {'patron': r'responsabilidad\s+extendida\s+del\s+productor|REP', 'fuente_nueva': 'ley', 'numero_nuevo': '20920', 'año_nuevo': '2016'}
]
reglas_administrativo = [
    {'patron': r'ley\s+de\s+bases\s+de\s+los\s+procedimientos\s+administrativos|ley\s+19880', 'fuente_nueva': 'ley', 'numero_nuevo': '19880', 'año_nuevo': '2003'},
    {'patron': r'ley\s+org[aá]nica\s+constitucional\s+de\s+municipalidades|ley\s+18695', 'fuente_nueva': 'ley', 'numero_nuevo': '18695', 'año_nuevo': '1988'},
    {'patron': r'estatuto\s+administrativo', 'fuente_nueva': 'ley', 'numero_nuevo': '18834', 'año_nuevo': '1989'},
]
reglas_laboral = [
    {'patron': r'ley\s+de\s+accidentes\s+del\s+trabajo\s+y\s+enfermedades\s+profesionales|ley\s+16744', 'fuente_nueva': 'ley', 'numero_nuevo': '16744', 'año_nuevo': '1968'},
    {'patron': r'ley\s+de\s+isapres', 'fuente_nueva': 'decreto con fuerza de ley', 'numero_nuevo': '1', 'año_nuevo': '2005'},
]
reglas_ambiente = [
    {'patron': r'ley\s+sobre\s+bases\s+generales\s+del\s+medio\s+ambiente|lbgma|ley\s+19300', 'fuente_nueva': 'ley', 'numero_nuevo': '19300', 'año_nuevo': '1994'},
    {'patron': r'sistema\s+de\s+evaluaci[oó]n\s+de\s+impacto\s+ambiental|seia', 'fuente_nueva': 'ley', 'numero_nuevo': '19300', 'año_nuevo': '1994'},
    {'patron': r'ley\s+org[aá]nica\s+de\s+la\s+superintendencia\s+del\s+medio\s+ambiente|losma|ley\s+20417', 'fuente_nueva': 'ley', 'numero_nuevo': '20417', 'año_nuevo': '2010'},
]
reglas_codigos = [
    {'patron': r'c[oó]digo\s+del\s+trabajo', 'fuente_nueva': 'decreto con fuerza de ley', 'numero_nuevo': '1', 'año_nuevo': '2002'},
    {'patron': r'c[oó]digo\s+tributario', 'fuente_nueva': 'decreto ley', 'numero_nuevo': '830', 'año_nuevo': '1974'},
    {'patron': r'c[oó]digo\s+de\s+miner[ií]a|c[oó]digo\+minero', 'fuente_nueva': 'ley', 'numero_nuevo': '18248', 'año_nuevo': '1983'},
]

todas_las_reglas = (
    reglas_normativas + 
    reglas_administrativo + 
    reglas_laboral + 
    reglas_ambiente + 
    reglas_codigos
)

df_procesado = aplicar_reglas_unificadas(df_procesado, todas_las_reglas)


In [ ]:
condicion = (df_procesado['numero_fuente_ajustado'] == 9725)  | (df_procesado['numero_fuente_ajustado'] == 72567)  | (df_procesado['numero_fuente_ajustado'] == 725) 
df_procesado.loc[condicion, 'año_fuente'] = 1967
df_procesado.loc[condicion, 'fuente'] = "decreto con fuerza de ley"
df_procesado.loc[condicion, 'numero_fuente_ajustado'] = 725

condicion = (df_procesado['numero_fuente_ajustado'] == 9594) | (df_procesado['numero_fuente_ajustado'] == 59499) | (df_procesado['numero_fuente_ajustado'] == 549) 
df_procesado.loc[condicion, 'año_fuente'] = 1999
df_procesado.loc[condicion, 'fuente'] = "decreto supremo"
df_procesado.loc[condicion, 'numero_fuente_ajustado'] = 594

condicion = (df_procesado['numero_fuente_ajustado'] == 9144 ) | (df_procesado['numero_fuente_ajustado'] == 14461) | (df_procesado['numero_fuente_ajustado'] == 144) 
df_procesado.loc[condicion, 'año_fuente'] = 1961
df_procesado.loc[condicion, 'fuente'] = "decreto supremo"
df_procesado.loc[condicion, 'numero_fuente_ajustado'] = 144

condicion = (df_procesado['numero_fuente_ajustado'] == 9484 ) | (df_procesado['numero_fuente_ajustado'] == 48490) | (df_procesado['numero_fuente_ajustado'] == 484) 
df_procesado.loc[condicion, 'año_fuente'] = 1990
df_procesado.loc[condicion, 'fuente'] = "decreto supremo"
df_procesado.loc[condicion, 'numero_fuente_ajustado'] = 484

condicion = (df_procesado['numero_fuente_ajustado'] == 1 ) & (df_procesado['año_fuente'] == 2007 )  
df_procesado.loc[condicion, 'año_fuente'] = 2009
df_procesado.loc[condicion, 'fuente'] = "decreto con fuerza de ley"

condicion = (df_procesado['numero_fuente_ajustado'] == 1 ) & (df_procesado['año_fuente'] == 1989 ) 
df_procesado.loc[condicion, 'año_fuente'] = 1990
df_procesado.loc[condicion, 'fuente'] = "decreto con fuerza de ley"

condicion = (df_procesado['numero_fuente_ajustado'] == 1518 ) & (df_procesado['año_fuente'] == 2013 ) 
df_procesado.loc[condicion, 'fuente'] = "resolucion"

In [ ]:
df_procesado['Tipologia_CE'] = df_procesado['Tipologia_CE'].str.replace('Clasificación Fallida', 'No Clasificado')
df_procesado['Tipologia_CE'] = df_procesado['Tipologia_CE'].str.replace('No Clasificable', 'No Clasificado')
df_procesado['Tipologia_CE'].fillna('No Clasificado', inplace=True)

df_procesado['Subtipologia_CE'] = df_procesado['Subtipologia_CE'].str.replace('El modelo no pudo generar una respuesta válida tras varios intentos.', 'No Clasificado')
df_procesado['Subtipologia_CE'] = df_procesado['Subtipologia_CE'].str.replace('Descripción General o Documental', 'No Clasificado')
df_procesado['Subtipologia_CE'].fillna('No Clasificado', inplace=True)

In [ ]:
# Guardar el DataFrame actualizado
df_procesado.to_excel(os.path.join(path, "obligaciones_combinadas_expandidas_ajustadas.xlsx"), index=False)

In [ ]:
df_procesado.loc[df_procesado['seccion'] == 'pas', 'fuente'] = 'permisos ambientales sectoriales'

In [ ]:
df_procesado.to_excel(r"RUTA\obligaciones_combinadas_expandidas_ajustadas.xlsx", index=False)